# Code analyzers

In [ ]:
file_path = "./examples/before/1.py"

## Flake8

In [ ]:
flake8_out = !flake8 --format='%(row)d,%(col)d::%(code)s::%(text)s' "$file_path"

for i in flake8_out:
    print(i)

flake8_out = '\n'.join(flake8_out)

## mypy

In [ ]:
mypy_out = !mypy "$file_path" --no-color-output --pretty --strict

for i in mypy_out:
    print(i)

mypy_out = '\n'.join(mypy_out)

## Pylint

In [ ]:
# pylint_out = !pylint $file_path --msg-template='{line}:{column}: {msg_id}: {msg} ({symbol})'
#
# for i in pylint_out:
#     print(i)
#
# pylint_out = '\n'.join(pylint_out)

The magic command doesn't see the variable file path.

## Black

In [ ]:
# $ black examples/1.py
# reformatted examples/1.py
#
# All done! ✨ 🍰 ✨
# 1 file reformatted.

It just refactors the code seemingly according to flake8 suggestions.

# Refactoring

## Read the source code

In [ ]:
with open(file_path, 'r') as f:
    code = f.read()

for i in code.split('\n'):
    print(i)

## Import LLM

In [4]:
from transformers import set_seed, AutoTokenizer, AutoModelForCausalLM

set_seed(123)

In [ ]:
# tokenizer = AutoTokenizer.from_pretrained("BarraHome/Mistroll-7B-v2.2")
# model = AutoModelForCausalLM.from_pretrained("BarraHome/Mistroll-7B-v2.2")

tokenizer = AutoTokenizer.from_pretrained("pansophic/rocket-3B")
model = AutoModelForCausalLM.from_pretrained("pansophic/rocket-3B")

# tokenizer = AutoTokenizer.from_pretrained("TinyLlama/TinyLlama-1.1B-step-50K-105b")
# model = AutoModelForCausalLM.from_pretrained("TinyLlama/TinyLlama-1.1B-step-50K-105b")

device = "cuda"
# device = "mps"
model.to(device)

In [6]:
prompt = "Hey, are you conscious? Can you talk to me?"
model_inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

In [7]:
generated_ids = model.generate(**model_inputs, max_length=512, do_sample=True)
responses = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
responses[len(prompt):]

## Prompt LLM

In [8]:
prompt = "You are refactoring assistant. Your task is to rewrite the provided code based on provided hints to clean code without errors.\n" + "Hints:\n" + "format: column::error_code::error:\n" + flake8_out + "\n" + mypy_out + "\n" + "Code:\n" + code + "\n" + "Make you answer strict and clean and include only rewrited code."
for i in prompt.split('\n'):
    print(i)
model_inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

In [9]:
generated_ids = model.generate(**model_inputs, max_length=512, do_sample=True)
responses = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

In [10]:
for i in responses[len(prompt) + 1:].split('\n'):
    print(i)